In [1]:
#probably don't need this
import gc
gc.collect()

0

In [2]:
#i think you'll have to move this file onto hpc first? 
import dask.dataframe as dd
ddf = dd.read_json("goodreads_reviews_dedup.json",lines=True,nrows=10000)
df = ddf.compute()

In [4]:
df.head() #make sure it loaded

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,24375664,5cd416f3efc3f944fce4ce2db2290d5e,5,Mind blowingly cool. Best science fiction I've...,Fri Aug 25 13:55:02 -0700 2017,Mon Oct 09 08:55:59 -0700 2017,Sat Oct 07 00:00:00 -0700 2017,Sat Aug 26 00:00:00 -0700 2017,16,0
1,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
2,8842281e1d1347389f2ab93d60773d4d,6392944,5e212a62bced17b4dbe41150e5bb9037,3,I haven't read a fun mystery book in a while a...,Mon Jul 24 02:48:17 -0700 2017,Sun Jul 30 09:28:03 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,Mon Jul 24 00:00:00 -0700 2017,6,0
3,8842281e1d1347389f2ab93d60773d4d,22078596,fdd13cad0695656be99828cd75d6eb73,4,"Fun, fast paced, and disturbing tale of murder...",Mon Jul 24 02:33:09 -0700 2017,Sun Jul 30 10:23:54 -0700 2017,Sun Jul 30 15:42:05 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,22,4
4,8842281e1d1347389f2ab93d60773d4d,6644782,bd0df91c9d918c0e433b9ab3a9a5c451,4,A fun book that gives you a sense of living in...,Mon Jul 24 02:28:14 -0700 2017,Thu Aug 24 00:07:20 -0700 2017,Sat Aug 05 00:00:00 -0700 2017,Sun Jul 30 00:00:00 -0700 2017,8,0


In [3]:
#all required imports
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [4]:
#data cleaning
df = df[df['rating'] != 0]
df.reset_index(drop=True, inplace=True)
df = df[['review_text','rating']]
df.head()

,review_text,rating
0,Mind blowingly cool. Best science fiction I've...,5
1,This is a special book. It started slow for ab...,5
2,I haven't read a fun mystery book in a while a...,3
3,"Fun, fast paced, and disturbing tale of murder...",4
4,A fun book that gives you a sense of living in...,4


In [6]:
#rest of this came from the wine medium article

# Download necessary resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Text preprocessing function
def preprocess_text(text: str) -> str:
    # Text cleaning
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Stop word removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
df['review_text_tokenized'] = df['review_text'].apply(preprocess_text)

In [9]:
# Tokenize the text and convert to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['review_text_tokenized'])
sequences = tokenizer.texts_to_sequences(df['review_text_tokenized'])

# Pad sequences to a fixed length
max_sequence_length = 100  # Maximum sequence length to pad
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2)

In [17]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_sequence_length))
model.add(LSTM(64))
model.add(Dropout(0.1))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=True)

# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")

Epoch 1/10
241/241 [==============================] - 41s 144ms/step - loss: 1.9015 - val_loss: 0.9789
Epoch 2/10
241/241 [==============================] - 32s 135ms/step - loss: 0.7647 - val_loss: 0.8471
Epoch 3/10
241/241 [==============================] - 32s 132ms/step - loss: 0.4553 - val_loss: 0.8396
Epoch 4/10
241/241 [==============================] - 32s 132ms/step - loss: 0.3298 - val_loss: 0.8546
Epoch 5/10
241/241 [==============================] - 34s 139ms/step - loss: 0.2445 - val_loss: 0.8698
Epoch 6/10
241/241 [==============================] - 33s 137ms/step - loss: 0.1962 - val_loss: 0.9202
Epoch 7/10
241/241 [==============================] - 32s 134ms/step - loss: 0.1705 - val_loss: 0.9026
Epoch 8/10
241/241 [==============================] - 33s 135ms/step - loss: 0.1486 - val_loss: 0.8987
Epoch 9/10
241/241 [==============================] - 32s 135ms/step - loss: 0.1286 - val_loss: 0.9149
Epoch 10/10
61/61 [==============================] - 4s 33ms/step
Mean Sq

In [18]:
model.save('trained_model/trained_model_on_first_10k.keras') #i think this will save it to hpc

In [19]:
#dont need this
vals = df['review_text'].tolist()
print(vals[0])

Mind blowingly cool. Best science fiction I've read in some time. I just loved all the descriptions of the society of the future - how they lived in trees, the notion of owning property or even getting married was gone. How every surface was a screen. 
 The undulations of how society responds to the Trisolaran threat seem surprising to me. Maybe its more the Chinese perspective, but I wouldn't have thought the ETO would exist in book 1, and I wouldn't have thought people would get so over-confident in our primitive fleet's chances given you have to think that with superior science they would have weapons - and defenses - that would just be as rifles to arrows once were. 
 But the moment when Luo Ji won as a wallfacer was just too cool. I may have actually done a fist pump. Though by the way, if the Dark Forest theory is right - and I see no reason why it wouldn't be - we as a society should probably stop broadcasting so much signal out into the universe.


In [23]:
df.head(2) #ignore

,review_text,rating,review_text_tokenized
0,Mind blowingly cool. Best science fiction I've...,5,"[mind, blowingly, cool, best, science, fiction..."
1,This is a special book. It started slow for ab...,5,"[special, book, started, slow, first, third, m..."


In [22]:
#this is to just make sure it worked 
from keras.models import load_model

# Load the trained model
loaded_model = load_model('trained_model/trained_model_on_first_10k.keras')

def predict_review_score(review_text: str) -> float:
    # Preprocess the review text
    sequence = tokenizer.texts_to_sequences([review_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length)

    # Make the prediction using the loaded model
    predicted_score = loaded_model.predict(padded_sequence)[0][0]

    return predicted_score

# Sample review to predict the score
sample_review = "Mind blowingly cool. Best science fiction I've read in some time. I just loved all the descriptions of the society of the future - how they lived in trees, the notion of owning property or even getting married was gone. How every surface was a screen. The undulations of how society responds to the Trisolaran threat seem surprising to me. Maybe its more the Chinese perspective, but I wouldn't have thought the ETO would exist in book 1, and I wouldn't have thought people would get so over-confident in our primitive fleet's chances given you have to think that with superior science they would have weapons - and defenses - that would just be as rifles to arrows once were. But the moment when Luo Ji won as a wallfacer was just too cool. I may have actually done a fist pump. Though by the way, if the Dark Forest theory is right - and I see no reason why it wouldn't be - we as a society should probably stop broadcasting so much signal out into the universe."
# Call the function to predict the score
predicted_score = predict_review_score(sample_review)
print(f"Predicted Score: {predicted_score:.2f}")

1/1 [==============================] - 1s 1s/step
Predicted Score: 4.35


### Ignore everything below this

In [2]:
pip install allennlp==0.4.2

Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for allennlp==0.4.2 from https://files.pythonhosted.org/packages/c5/60/fa613bdea022bd6c26176f5786efcc0b5a6d8acf97131e324179a99fcbaf/allennlp-0.4.2-py3-none-any.whl.metadata
  Using cached allennlp-0.4.2-py3-none-any.whl.metadata (1.3 kB)
INFO: pip is looking at multiple versions of allennlp to determine which version is compatible with other requirements. This could take a while.


ERROR: Ignored the following versions that require a different python version: 0.2.0 Requires-Python ==3.6
ERROR: Could not find a version that satisfies the requirement torch==0.3.1 (from allennlp) (from versions: 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==0.3.1


In [4]:
pip install torch==0.3.1

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch==0.3.1 (from versions: 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==0.3.1


In [13]:
pip install tensorflow-hub --user


  Obtaining dependency information for tensorflow-hub from https://files.pythonhosted.org/packages/e5/50/00dba77925bf2a0a1e45d7bcf8a69a1d2534fb4bb277d9010bd148d2235e/tensorflow_hub-0.16.1-py2.py3-none-any.whl.metadata
  Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl.metadata (1.3 kB)
  Obtaining dependency information for tf-keras>=2.14.1 from https://files.pythonhosted.org/packages/8a/ed/e08afca471299b04a34cd548e64e89d0153eda0e6cf9b715356777e24774/tf_keras-2.18.0-py3-none-any.whl.metadata
  Using cached tf_keras-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Obtaining dependency information for tensorflow<2.19,>=2.18 from https://files.pythonhosted.org/packages/cf/24/271e77c22724f370c24c705f394b8035b4d27e4c2c6339f3f45ab9b8258e/tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Obtaining dependency information for tensorflow-intel==2.18.0 from https://files.pythonhosted.org/packages/76/ad/fa6c508a15

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [12]:
import tensorflow_hub as hub
import tensorflow as tf

# Load the pretrained ELMo model
elmo = hub.load("https://tfhub.dev/google/elmo/3")

ModuleNotFoundError: No module named 'tensorflow_hub'

In [ ]:
def elmo_embed_sentence(sentence):
    # Convert sentence to a TensorFlow tensor
    embeddings = elmo.signatures['default'](tf.constant([sentence]))
    # Get the ELMo embedding (shape: [1, sequence_length, 1024])
    elmo_embeddings = embeddings['elmo']
    return elmo_embeddings[0].numpy()  # Convert to a NumPy array for easier handling

In [ ]:
df['elmo_embeddings'] = df['text_column'].apply(elmo_embed_sentence)